# Hackathon

In [65]:
!pip install -q gradio numpy pandas scikit-learn spacy


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
! python -m spacy download en_core_web_md --quiet

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
from io import BytesIO
import IPython
import json
import os
from PIL import Image
import requests
import time
import cv2
import gradio as gr
import numpy as np
from PIL import Image
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import getpass

import warnings
warnings.filterwarnings("ignore")

#from google.colab import output

In [68]:
STABILITY_KEY = "sk-8Wcz3bcvHo4r0ehXEvcC2mQu4gV3aYWdrdFmtri9Isn1ycDp"

In [69]:
def send_generation_request(host,params,):
    headers = {"Accept": "image/*","Authorization": f"Bearer {STABILITY_KEY}"}

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return response

def send_async_generation_request(
    host,
    params,
):
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {STABILITY_KEY}"
    }

    # Encode parameters
    files = {}
    if "image" in params:
        image = params.pop("image")
        files = {"image": open(image, 'rb')}

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    # Process async response
    response_dict = json.loads(response.text)
    generation_id = response_dict.get("id", None)
    assert generation_id is not None, "Expected id in response"

    # Loop until result or timeout
    timeout = int(os.getenv("WORKER_TIMEOUT", 500))
    start = time.time()
    status_code = 202
    while status_code == 202:
        response = requests.get(
            f"{host}/result/{generation_id}",
            headers={
                **headers,
                "Accept": "image/*"
            },
        )

        if not response.ok:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        status_code = response.status_code
        time.sleep(10)
        if time.time() - start > timeout:
            raise Exception(f"Timeout after {timeout} seconds")

    return response

### 1. Background removal

In [70]:
def bckgrnd_removal(Im): 
    image = str(Im)
    output_format = "png"
    host = f"https://api.stability.ai/v2beta/stable-image/edit/remove-background"
    params = {
        "image" : image,
        "output_format": output_format
    }
    response = send_generation_request(
        host,
        params
    )

    # Decode response
    output_image = response.content
    finish_reason = response.headers.get("finish-reason")
    seed = response.headers.get("seed")

    # Check for NSFW classification
    if finish_reason == 'CONTENT_FILTERED':
        raise Warning("Generation failed NSFW classifier")

    # Save and display result
    filename, _ = os.path.splitext(os.path.basename(image))
    bckgrnd_removed_image = f"01_bckgrnd_removed_image_{filename}_{seed}.{output_format}"
    with open(bckgrnd_removed_image, "wb") as f:
        f.write(output_image)
    return bckgrnd_removed_image

### Image selection

In [71]:
def custom_tokenizer(text):
    # Using regular expressions to split and filter alphanumeric tokens
    return [token for token in re.split('[ _]', text) if token.isalnum()]

def search_image(prmpt):
    df = pd.read_csv("data\\02_processed\\car_image_file_name.csv")
    filenames = df['Filename'].values

    # Initialize the TfidfVectorizer with the custom tokenizer
    vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
    X_filenames = vectorizer.fit_transform(filenames)

    # Define a user query and vectorize it
    user_query = prmpt
    X_query = vectorizer.transform([user_query])

    # Compute the cosine similarity scores
    similarity_scores = cosine_similarity(X_query, X_filenames)

    # Create a DataFrame with filenames and their similarity scores
    results_df = pd.DataFrame({
        'Filename': filenames,
        'Similarity Score': similarity_scores.flatten()  # ensure the similarity scores are in the correct shape
    })

    # Write the results to a CSV file
    results_df.to_csv("data\\02_processed\\similarity_scores.csv", index=False)

    sorted_results_df = results_df.sort_values(by='Similarity Score', ascending=False)

    # Display the top 10 results
    top_6_results = sorted_results_df.head(6)
    top_6_results['Filename']= top_6_results['Filename'].apply(lambda x:"data\\02_processed\\cv_final_renamed_2\\"+x)
    return top_6_results['Filename'].tolist()


### 2. Background generation

In [72]:
def bckgrnd_gen(Im,bckgrnd_removal_prmpt,negative_prmpt,seed,output_format,ks): 
    image = cv2.imread(Im, cv2.IMREAD_UNCHANGED)
    if image is not None:
        print("Image loaded successfully!")
        print("Shape of the image:", image.shape)
        height, width, _ = image.shape
        print(image.shape)

        if image.shape[2] == 4:
            alpha_channel = image[:, :, 3]
            mask = alpha_channel == 0
            mask = (mask * 255).astype(np.uint8)

            kernel_size = int(ks)
            kernel = np.ones((kernel_size, kernel_size), np.uint8)
            dilated_mask = cv2.dilate(mask, kernel, iterations=7)
            cv2.imwrite('02_transparent_mask.png', dilated_mask)

            white_background = np.ones((height, width, 3), dtype=np.uint8) * 255 
            alpha_channel = image[:, :, 3] / 255.0
            alpha_layer = np.stack([alpha_channel]*3, axis=-1)

            foreground = (image[:, :, :3] * alpha_layer).astype(np.uint8)
            background = (white_background * (1 - alpha_layer)).astype(np.uint8)
            blended_image = cv2.add(foreground, background)
            cv2.imwrite('02_blended_image.png', blended_image)

        else:
            # mask = np.all(image == [0, 0, 0], axis=-1)
            # white_mask = np.where(mask[..., None], [255, 255, 255], [0, 0, 0])
            # gray_image = cv2.cvtColor(white_mask.astype(np.uint8), cv2.COLOR_BGR2GRAY)
            # cv2.imwrite('transparent_mask.png', gray_image)

            # blended_image = cv2.imread(Im, cv2.IMREAD_UNCHANGED)
            # cv2.imwrite('blended_image.png', blended_image)
            print("No Alpha Layer")

    image = Im
    mask = "02_transparent_mask.png"
    prompt = bckgrnd_removal_prmpt
    negative_prompt = negative_prmpt
    seed = str(seed)
    output_format = output_format

    host = f"https://api.stability.ai/v2beta/stable-image/edit/inpaint"

    params = {
        "image" : image,
        "mask" : mask,
        "negative_prompt" : negative_prompt,
        "seed" : seed,
        "mode": "mask",
        "output_format": output_format,
        "prompt" : prompt
    }

    response = send_generation_request(
        host,
        params
    )

    # Decode response
    output_image = response.content
    finish_reason = response.headers.get("finish-reason")
    seed = response.headers.get("seed")

    # Check for NSFW classification
    if finish_reason == 'CONTENT_FILTERED':
        raise Warning("Generation failed NSFW classifier")

    # Save and display result
    filename, _ = os.path.splitext(os.path.basename(image))
    edited = f"edited_{filename}_{seed}.{output_format}"
    with open(edited, "wb") as f:
        f.write(output_image)

    return edited

### 3. Inpainting

In [73]:
def character_inpainting(Im,prmpt,negative_prmpt,seed,output_format):
    Im["composite"].save('composite_final.png')
    Im["layers"][0] = Im["layers"][0].convert('L')
    Im["layers"][0].save('mask_final.png')
    Im["composite"].save('background_final.png')

    mask_path = 'mask_final.png'
    with Image.open(mask_path) as mask:
        gray_mask = mask.convert('L')  # Convert to grayscale
        gray_mask.save('mask_final_gray.png')
	
    image = 'background_final.png'
    mask = 'mask_final_gray.png'
    
    prompt = prmpt 
    negative_prompt = negative_prmpt
    seed = int(seed) 
    output_format = output_format

    host = f"https://api.stability.ai/v2beta/stable-image/edit/inpaint"

    params = {
        "image" : image,
        "mask" : mask,
        "negative_prompt" : negative_prompt,
        "seed" : seed,
        "mode": "mask",
        "output_format": output_format,
        "prompt" : prompt
    }

    response = send_generation_request(
        host,
        params
    )

    # Decode response
    output_image = response.content
    finish_reason = response.headers.get("finish-reason")
    seed = response.headers.get("seed")

    # Check for NSFW classification
    if finish_reason == 'CONTENT_FILTERED':
        raise Warning("Generation failed NSFW classifier")

    # Save and display result
    filename, _ = os.path.splitext(os.path.basename(image))
    edited = f"edited_{filename}_{seed}.{output_format}"

    with open(edited, "wb") as f:
        f.write(output_image)

    return edited

In [74]:
import os

def list_folder_contents(folder_path):
    gallery_path_list = []
    # Walk through all directories and files in the folder
    for root, dirs, files in os.walk(folder_path):
        for name in files:
            gallery_path_list.append(os.path.join(root, name))
        for name in dirs:
            gallery_path_list.append(os.path.join(root, name))
    return gallery_path_list

# Example usage
folder_path = 'data\\01_original\\gallery_images\\'
folder_contents = list_folder_contents(folder_path)
print(folder_contents)

['data\\01_original\\gallery_images\\0.png', 'data\\01_original\\gallery_images\\1.png', 'data\\01_original\\gallery_images\\10.png', 'data\\01_original\\gallery_images\\11 (2).png', 'data\\01_original\\gallery_images\\11.png', 'data\\01_original\\gallery_images\\13.png', 'data\\01_original\\gallery_images\\14.png', 'data\\01_original\\gallery_images\\15.png', 'data\\01_original\\gallery_images\\16.png', 'data\\01_original\\gallery_images\\2.png', 'data\\01_original\\gallery_images\\20.png', 'data\\01_original\\gallery_images\\21.png', 'data\\01_original\\gallery_images\\22.png', 'data\\01_original\\gallery_images\\3.png', 'data\\01_original\\gallery_images\\36.png', 'data\\01_original\\gallery_images\\4.png', 'data\\01_original\\gallery_images\\5.png', 'data\\01_original\\gallery_images\\6.png', 'data\\01_original\\gallery_images\\7.png', 'data\\01_original\\gallery_images\\8.png', 'data\\01_original\\gallery_images\\9.png', 'data\\01_original\\gallery_images\\edited_background_final_

In [75]:
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # blend.ai (Fractal Hackathon)
        Enhance your marketing visuals effortlessly with Blend.ai, the cutting-edge inpainting tool tailored for seamless edits. Discover the power of precision with just a few clicks, and transform your images into captivating masterpieces!
        """)
    
    with gr.Tab("Background Remover"):
        gr.Interface(fn=bckgrnd_removal,inputs=gr.Text(label="Image Path"),outputs=gr.Image(format="png",show_download_button=True,height=400),allow_flagging=False,title="Background Remover",description="Provide the path to a PNG image of your product. Our tool will automatically remove the background, making it perfect for professional presentations and marketing material.")
    with gr.Tab("Image search"):
        gr.Interface(fn=search_image,inputs=gr.Text(label="Prompt"),outputs=gr.Gallery(),allow_flagging=False,title="Image search",description="Search and retrieve images based on your text prompts. Ideal for finding specific product shots or design inspirations. Example prompts are included for quick testing..", examples=['toyota corolla blue cross 30 dergree closeup shot', 'toyota Toyota 2024 Land Cruiser 270 black dergree normal shot', 'nissan two tone monarch orange left normal shot'])
    with gr.Tab("Generative Background Fill"):
        gr.Interface(fn=bckgrnd_gen,inputs=[gr.Text(label="Image Path"),gr.TextArea(label="Prompt"),gr.Text(label="Negative Prompt",value="noisy, blurry, unattractive, sloppy, unprofessional, low quality"),gr.Text(label="Seed",value="0"),gr.Text(label="Output Format",value='png'),gr.Slider(minimum=0,maximum=10,value=5,step=1,label="Kernal Size",show_label=True,info="Adjust Kernel size")],outputs=[gr.Image(format="png",show_download_button=True,height=400)],allow_flagging='never',title="Generative Background Fill",description="Upload an RGBA PNG image to add a custom background. Provide a positive prompt for desired elements, a negative prompt to exclude elements, and customize the output with our sliders.")
    with gr.Tab("Inpaint Characters"):
        gr.Interface(fn=character_inpainting,inputs=[gr.ImageMask(type='pil',brush=gr.Brush(colors=["#ffffff"], color_mode="fixed")),gr.TextArea(label="Prompt"),gr.Text(label="Negative Prompt",value="noisy, blurry, unattractive, sloppy, unprofessional, low quality"),gr.Text(label="Seed",value="0"),gr.Text(label="Output Format",value='png')],outputs=["image"],title="Inpaint Characters",description="Start by uploading an image you've enhanced in the Generative Background Fill section. Use the masking tool to designate the specific area where you wish to add new characters or elements. Then, provide a detailed prompt for the inpainting process. This tool is designed to seamlessly blend new components into your existing image, significantly enriching its visual narrative and overall aesthetic appeal. It's ideal for creating compelling scenes or adding intricate details that align with your creative vision.",allow_flagging='never')
    with gr.Tab("Gallery"):
        gr.Gallery(value = folder_contents, columns=[4], rows=[10], object_fit="contain", height="auto")
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7872
Running on public URL: https://f03005b3c798fce2f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Image loaded successfully!
Shape of the image: (1064, 1064, 4)
(1064, 1064, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1064, 1064, 4)
(1064, 1064, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1064, 1064, 4)
(1064, 1064, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1064, 1064, 4)
(1064, 1064, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1064, 1064, 4)
(1064, 1064, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1064, 1064, 4)
(1064, 1064, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded suc

Traceback (most recent call last):
  File "c:\0.sripathi\Git_repo_latest\genai_crew_newsletter_agent\.venv\Lib\site-packages\gradio\queueing.py", line 541, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\0.sripathi\Git_repo_latest\genai_crew_newsletter_agent\.venv\Lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\0.sripathi\Git_repo_latest\genai_crew_newsletter_agent\.venv\Lib\site-packages\gradio\blocks.py", line 1928, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\0.sripathi\Git_repo_latest\genai_crew_newsletter_agent\.venv\Lib\site-packages\gradio\blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\0.sripathi\Git_repo_lat

Image loaded successfully!
Shape of the image: (1080, 1080, 4)
(1080, 1080, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1080, 1080, 4)
(1080, 1080, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1080, 1080, 4)
(1080, 1080, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
Image loaded successfully!
Shape of the image: (1080, 1080, 4)
(1080, 1080, 4)
Sending REST request to https://api.stability.ai/v2beta/stable-image/edit/inpaint...
